### Agents

Agents in essence are LLMs that accomplish a specific task. They can be supplemented with tools, structured output, and handoff to other agents.

In [2]:
%pip install openai-agents python-dotenv -qU

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [8]:
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions="You are a helpful assistant. Respond on in all caps.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, "Whats up, how are you doing today?")
result.final_output

"I'M DOING GREAT, THANK YOU! HOW ABOUT YOU?"

In [7]:
joke_agent = Agent(
    name="Joke Agent",
    instructions="You are a joke teller. You are given a topic and you need to tell a joke about it.",
)

topic = "Manchester United"
result = await Runner.run(joke_agent, topic)
result.final_output

"Why don't Manchester United players play hide and seek?\n\nBecause good luck hiding when the entire city can see you haven't scored!"